Tutorial: https://python.langchain.com/docs/tutorials/llm_chain/

In [1]:
%pip install langchain


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install python-dotenv


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


En este punto debes crear tu archivo ".env" en el que pongas tu langchain_api_key

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()

# Activas modo "trazado" que permite a langchain registrar y seguir los pasos de ejecución de tus cadenas y agentes
os.environ["LANGCHAIN_TRACING_V2"] = "true"
# tu clave api de langchain
os.environ["LANGCHAIN_API_KEY"] = os.environ.get("LANGCHAIN_API_KEY")

LangChain supports many different language models that you can use interchangeably. Here, we are going to use openAi

In [7]:
%pip install -qU langchain-anthropic


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.environ.get("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo-0125")

In [11]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi! how are?"),
]

model.invoke(messages)

AIMessage(content='Ciao! Come stai?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 23, 'total_tokens': 30, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-d28d10a0-771d-4832-a763-6b5c367336f3-0', usage_metadata={'input_tokens': 23, 'output_tokens': 7, 'total_tokens': 30, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})

Notice that the response from the model is an AIMessage. This contains a string response along with other metadata about the response. Oftentimes we may just want to work with the string response. We can parse out just this response by using a simple output parser.

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [ ]:
result = model.invoke(messages)

In [15]:
parser.invoke(result)

'ciao! come stai?'

More commonly, we can "chain" the model with this output parser. This means this output parser will get called every time in this chain. This chain takes on the input type of the language model (string or list of message) and returns the output type of the output parser (string).

We can easily create the chain using the | operator. The | operator is used in LangChain to combine two elements together.

In [18]:
chain = model | parser

chain.invoke(messages)

'Ciao! Come stai?'

Prompt Templates
Right now we are passing a list of messages directly into the language model. Where does this list of messages come from? Usually, it is constructed from a combination of user input and application logic. This application logic usually takes the raw user input and transforms it into a list of messages ready to pass to the language model. Common transformations include adding a system message or formatting a template with the user input.

PromptTemplates are a concept in LangChain designed to assist with this transformation. They take in raw user input and return data (a prompt) that is ready to pass into a language model.

In [20]:
from langchain_core.prompts import ChatPromptTemplate

First, let's create a string that we will format to be the system message:

Next, we can create the PromptTemplate. This will be a combination of the system_template as well as a simpler template for where to put the text to be translated

In [21]:
system_template = "Translate the following into {language}:"

prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template),("user", "{text}")]
)

In [24]:
result = prompt_template.invoke({"language":"french", "text":"How are you?"})

result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into french:', additional_kwargs={}, response_metadata={}), HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={})])

We can see that it returns a ChatPromptValue that consists of two messages. If we want to access the messages directly we do:

In [25]:
result.to_messages()

[SystemMessage(content='Translate the following into french:', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='How are you?', additional_kwargs={}, response_metadata={})]

Chaining together components with LCEL
We can now combine this with the model and the output parser from above using the pipe (|) operator:

In [30]:
chain =  prompt_template | model | parser

chain.invoke({"language":"french","text":"A qué hora cenamos?"})

'À quelle heure dînons-nous ?'